# Noiseness analysis

In [ ]:
import requests



In [ ]:
BASE_URL = 'http://localhost:4242'
POINTS_COUNT = 50
MAX_SELECTED_CELLS = 10
SOLUTION_DIM = POINTS_COUNT * 2 + MAX_SELECTED_CELLS * 2 + 1 
TRACK_SIZE_RANGE = (2, 6)
LENGTH_RANGE = (400, 2000)
ITERATIONS = 500
ARCHIVE_DIM = 3
INIT_POPULATION = ARCHIVE_DIM * ARCHIVE_DIM 

DEBUG_CROSSOVER = True
DEBUG_MUTATION = True

In [ ]:
def evaluate_solution_full(sol):
    solution_id = sol.get("id", 0)
    try:
        response = requests.post(
            f"{BASE_URL}/evaluate",
            json=sol,
            timeout=60
        )
        response.raise_for_status()
        fitness = response.json().get("fitness", {})
        return solution_id, True, "", fitness
    except (requests.RequestException, ValueError) as e:
        return solution_id, False, str(e), {}


In [ ]:
def generate_solution_fixed(track_id, track_size):
    try:
        response = requests.post(
            f"{BASE_URL}/generate",
            json={
                "id": track_id,
                "mode": "voronoi",
                "trackSize": track_size
            },
            timeout=60
        )
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error generating fixed solution for track {track_id}: {e}")
        return None


In [ ]:
def run_noisiness_full_for_track(track_id, track_size, num_iterations=10):
    results = []
    for i in range(num_iterations):
        sol = generate_solution_fixed(track_id, track_size)
        if sol is None:
            continue
        sol_id, success, msg, fitness = evaluate_solution_full(sol)
        if not success:
            print(f"Evaluation failed for track {track_id} iteration {i}: {msg}")
            continue
        record = {"track_id": track_id, "iteration": i}
        record.update(fitness)
        results.append(record)
    return results


In [ ]:
all_results = []
num_tracks = 100
num_iterations = 10
fixed_track_size = 4

for t in range(1, num_tracks + 1):
    track_results = run_noisiness_full_for_track(t, fixed_track_size, num_iterations)
    if track_results:
        all_results.extend(track_results)

df_features = pd.DataFrame(all_results)
print(df_features.head())


In [ ]:
metrics = [col for col in df_features.columns if col not in ["track_id", "iteration"]]
stats = df_features.groupby("track_id")[metrics].agg(["mean", "std"]).reset_index()
# Flatten multi-index columns
stats.columns = ["_".join(col).strip("_") for col in stats.columns.values]
print(stats.head())

In [ ]:
for metric in ["speed_entropy", "gaps_mean", "curvature_entropy"]:
    plt.figure(figsize=(10, 4))
    mean_col = f"{metric}_mean"
    std_col = f"{metric}_std"
    plt.errorbar(stats["track_id"], stats[mean_col], yerr=stats[std_col], fmt="o", capsize=3)
    plt.xlabel("Track ID")
    plt.ylabel(metric)
    plt.title(f"Noisiness Analysis: {metric}")
    plt.show()